## Word2Vec

#### 완전연결계층에 의한 변환
- 7개의 입력이 3개의 은닉층 뉴런에 대응되는 예
- 간단한 예를 위해 가중치만 생각하고 편향은 생략함
- 완전연결계층에 의한 변환의 파이썬 표현
 - 행렬곱은 np.matmul()로 구현
 - 입력 데이터 c는 2차원 데이터임 --> 추후 미니배치(여러 단어를 한꺼번에 학습) 처리를 고려함 

In [1]:
import numpy as np

c = np.array([[1,0,0,0,0,0,0]])    # 입력데이터, 2차원, 추후 미니배치위한 확장성고려
W = np.random.randn(7,3)           # 7*3 가중치행렬 랜덤
h = np.matmul(c, W)                # 은닉 노드

print(h)

[[-1.70089616 -0.54695342  0.29417508]]


In [2]:
import numpy as np

c = np.array([[1,0,0,0,0,0,0],[0,0,1,0,0,0,0]])    
W = np.random.randn(7,3)          
h = np.matmul(c, W)               # [1,0,0,0,0,0,0]의 은닉, [0,0,1,0,0,0,0]의 은닉 구함

print(h)

[[ 1.34751575 -0.05031772 -1.55552563]
 [-1.32448729 -0.08027747  1.52764905]]


In [3]:
class MatMul:
      def __init__(self, W): #초기화
          self.params = [W]
          self.grads = [np.zeros_like(W)]
          self.x = None

      def forward(self, x): # 순방향 추론
          W, = self.params
          out = np.dot(x, W)
          self.x = x
          return out

      def backward(self, dout): #역방향 기능
          W, = self.params
          dx = np.dot(dout, W.T)
          dW = np.dot(self.x.T, dout)
          print(dW, self.grads[0][...])
          self.grads[0][...] = dW
          print(self.grads[0])
          return dx

In [4]:
import numpy as np

c = np.array([[1,0,0,0,0,0,0]])    # 입력
W = np.random.randn(7,3)           # 가중치 랜덤
layer = MatMul(W)                  # W 가중치를 사용하여 layer 객체 생성
h = layer.forward(c)               # layer 객체의 순전파 수행 결과 --> 은닉 노드
print(h)
h = layer.backward(h)
print(h)

[[-0.67144698  0.65719715 -0.2256524 ]]
[[-0.67144698  0.65719715 -0.2256524 ]
 [-0.          0.         -0.        ]
 [-0.          0.         -0.        ]
 [-0.          0.         -0.        ]
 [-0.          0.         -0.        ]
 [-0.          0.         -0.        ]
 [-0.          0.         -0.        ]] [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[-0.67144698  0.65719715 -0.2256524 ]
 [-0.          0.         -0.        ]
 [-0.          0.         -0.        ]
 [-0.          0.         -0.        ]
 [-0.          0.         -0.        ]
 [-0.          0.         -0.        ]
 [-0.          0.         -0.        ]]
[[ 0.93366814 -1.06815424  1.09068492 -0.94194162  0.40194778 -0.25414866
  -2.0250053 ]]


In [5]:
[np.zeros_like(W)][0][...]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

####  CBOW 모델을 통한 추론

In [6]:
import numpy as np

# 샘플 맥락 데이터
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]]) #input  좌측
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]]) #input  우측

# 가중치 초기화
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# 계층 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)           # 두 입력의 출력인 hidden 값에 대한 평균

s = out_layer.forward(h)

print(s)

[[ 1.08177886 -1.26480394 -0.97266214  1.97584378  1.8306013  -1.17231628
  -0.36268982]]


#### 말뭉치 텍스트를 단어 ID로 변환
- 미리 구현한 preprocess() 함수 사용

In [7]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id) # int 값
            word_to_id[word] = new_id 
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

In [8]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

print(corpus)
print(word_to_id)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


#### corpus로부터 맥락과 타겟 생성


- create_contexts_target() 함수 정의
 - 입력: corpus, window_size (윈도우 크기가 1이면 좌우 한 단어씩 맥락에 포함)
 - 반환: 맥락, 타겟 (각각 넘파이 다차원 배열)

In [9]:
def create_contexts_target(corpus, window_size=1):

    target = corpus[window_size:-window_size]
    contexts = []

    for idx in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size + 1):
            if t == 0:
                continue
            cs.append(corpus[idx + t])
        contexts.append(cs)

    return np.array(contexts), np.array(target)

In [10]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

contexts, target = create_contexts_target(corpus, window_size = 1)
#
print(corpus)
print(id_to_word)
print(contexts)
print(target)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}
[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]
[1 2 3 4 1 5]


#### 맥락과 타겟의 one-hot 표현

- convert_one_hot() 함수 사용
 - 입력: corpus, vocab_size (어휘 수)
 - 반환: 입력받은 corpus의 one-hot 표현 (각각 넘파이 다차원 배열)

In [11]:
def convert_one_hot(corpus, vocab_size):
  N = corpus.shape[0]
  # 전체 문장의 길이 
  if corpus.ndim == 1: # Label
      one_hot = np.zeros((N, vocab_size), dtype=np.int32) # 7 X len(Vocab) or Vocab.nunique()
      for idx, word_id in enumerate(corpus):
          one_hot[idx, word_id] = 1  # 순서별로 몇번째 word 가 나왔는지

  elif corpus.ndim == 2:  # Sentence , Input 이 쌍으로 들어가고 사이 단어를 예측하는 것이기 때문
      C = corpus.shape[1]
      one_hot = np.zeros((N, C, vocab_size), dtype=np.int32)
      for idx_0, word_ids in enumerate(corpus):
          for idx_1, word_id in enumerate(word_ids):
              one_hot[idx_0, idx_1, word_id] = 1

  return one_hot

#### 학습 데이터 준비의 전체 과정

In [12]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

contexts, target = create_contexts_target(corpus, window_size = 1)

vocab_size = len(word_to_id)

contexts = convert_one_hot(contexts, vocab_size)
target = convert_one_hot(target, vocab_size)

print(contexts)
print(target)

[[[1 0 0 0 0 0 0]
  [0 0 1 0 0 0 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 1 0 0 0]]

 [[0 0 1 0 0 0 0]
  [0 0 0 0 1 0 0]]

 [[0 0 0 1 0 0 0]
  [0 1 0 0 0 0 0]]

 [[0 0 0 0 1 0 0]
  [0 0 0 0 0 1 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 0 0 0 1]]]
[[0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]]


#### CBOW 모델을 통한 추론

In [13]:
# 샘플 맥락 데이터
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]]) #input  좌측
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]]) #input  우측

# 가중치 초기화
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# 계층 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)           # 두 입력의 출력인 hidden 값에 대한 평균

s = out_layer.forward(h)

print(s)

[[-1.52918199  2.12135303  2.46362898 -0.43470271  0.59349073 -3.57326261
  -1.50627271]]


#### SimpleCBOW 클래스의 구현 
- common/SimpleCBOW.py
- init, forward, backword 메서드로 구성

- 초기화 메서드
 - 인수로 어휘 수(vocab_size)와 은닉층 뉴런 수(hidden_size)를 받음
 - 가중치 2개 생성 (W_in, W_out)
 - 입력 MatMul 계층 2개, 출력 MatMul 계층 1개, Softmax with Loss 계층 1개 생성
   - 입력 MatMul 계층은 맥락에서 사용하는 단어 수만큼 생성
 - 신경망에서 사용되는 모든 매개변수와 기울기를 모아두기 위한 params와 grads 리스트 생성
 - word_vecs에 단어 가중치 (즉 단어 벡터) 저장

In [14]:
class SimpleCBOW:
    def __init__(self, vocab_size, hidden_size):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(H, V).astype('f')

        # 계층 생성
        self.in_layer0 = MatMul(W_in)
        self.in_layer1 = MatMul(W_in)
        self.out_layer = MatMul(W_out)
        self.loss_layer = SoftmaxWithLoss()

        # 모든 가중치와 기울기를 리스트에 모은다.
        layers = [self.in_layer0, self.in_layer1, self.out_layer]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in
        

- 순전파 메서드

In [15]:
def forward(self, contexts, target):
    h0 = self.in_layer0.forward(contexts[:, 0])
    h1 = self.in_layer1.forward(contexts[:, 1])
    h = (h0 + h1) * 0.5
    score = self.out_layer.forward(h)
    loss = self.loss_layer.forward(score, target)
    
    return loss

- 역전파 메서드

In [16]:
def backward(self, dout=1):
    ds = self.loss_layer.backward(dout)
    da = self.out_layer.backward(ds)
    da *= 0.5
    self.in_layer1.backward(da)
    self.in_layer0.backward(da)

    return None

### WV Example

In [17]:
import gensim.downloader as api # 1.6GB
wv = api.load('word2vec-google-news-300')

[===================================---------------] 71.7% 1191.8/1662.8MB downloaded


KeyboardInterrupt



In [ ]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

In [ ]:
vec_king = wv['king']

In [ ]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

### Word2Vec 훈련시키기

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec

In [ ]:
w2v = word2vec.Word2Vec(sentences = w2v_input,
                        vector_size = 30,
                        window = 3,
                        min_count = 1,
                        sg = 1).wv

In [ ]:
wv = w2v.wv

In [ ]:
train_mean_vector = []

for words in tqdm(train_data):
        tmp = np.zeros(30)             # 다음 customer ID에 대한 vector를 계삲하기 전 0으로 초기화
        cnt = 0
        for word in words:
            try:
                tmp += w2v[word]
                cnt += 1
            except:
                pass
        tmp /= cnt                      # customer ID 에 있는 아이템 갯수로 전체 벡터를 mean해줌  
        train_mean_vector.append(tmp)
        
train_mean_vector = np.array(train_mean_vector)

## K-means

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
 
np.random.seed(100)
num_data = 50
 
x11 = np.linspace(0.3,0.7,20)
x12 = np.linspace(1.3,1.8,15)
x13 = np.linspace(2.4,3,15)
x1 = np.concatenate((x11,x12,x13),axis=None)
error = np.random.normal(1,0.5,num_data)
x2 = 1.5*x1+2+error
fig = plt.figure(figsize=(7,7))
fig.set_facecolor('white')
plt.scatter(x1, x2, color='k')
plt.show()

In [ ]:
print(X[1,:])

In [ ]:
def kmeans_clustering(X, n_clusters, init_center=None, max_iter=10, epsilon=1e-4, random_state=100): 
    # inititalize centeroids
    if init_center is None:
        random.seed(random_state)
        idx = random.sample(range(X.shape[0]), n_clusters)
        center = X[idx,:]
    else:
        center = init_center
    iteration = 1
    # Center 지정 random하게
    
    labels_history = [] # label history 
    center_history = [] # centeroid history
    while(iteration<=max_iter):
        ## assign label
        labels = []
        for i in range(X.shape[0]):
            data = X[i, :]
            labels.append(np.argmin([np.linalg.norm(data-x) for x in center]))
            # 가장 거리가 짧은 센터로의 할당
        labels = np.array(labels)
        ## update centeroids
        next_center = []
        print(labels,': Labels')
        for i in range(n_clusters):
            target_idx = np.where(labels==i)[0] # Label 이 i 인 데이터
            center_val = np.mean(X[target_idx,:], axis=0) # 각 라벨의 좌표값들의 총 합, 평균이 구해진다. (X,Y 축 각각)
            print(center_val,'중심평균값')
            next_center.append(center_val)
 
        next_center = np.array(next_center) # 각 평균값이 이동함.
        if epsilon:
            if np.linalg.norm(next_center-center) <= epsilon:
                break
        center = next_center
        labels_history.append(labels)
        center_history.append(center)
        iteration += 1
    return (labels, iteration, labels_history, center_history)

In [ ]:
X = np.stack([x1, x2], axis=1)
 
init_center= np.array([[2,4],[1,5],[2.5,6]])
max_iter=50
epsilon=1e-10
random_state=101
n_clusters=3
results = kmeans_clustering(X, n_clusters, init_center, max_iter, epsilon=1e-4, 
                           random_state=100)
labels = results[0]

In [ ]:
fig = plt.figure(figsize=(7,7))
fig.set_facecolor('white')
for i, label in enumerate(labels):
    if label == 0:
        color = 'blue'
    elif label ==1:
        color = 'red'
    else:
        color = 'green'
    plt.scatter(X[i,0],X[i,1], color=color)
    
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

In [ ]:
labels_history = results[2]
for j, labels in enumerate(labels_history):
    fig = plt.figure(figsize=(7,7))
    fig.set_facecolor('white')
    for i, label in enumerate(labels):
        if label == 0:
            color = 'blue'
        elif label ==1:
            color = 'red'
        else:
            color = 'green'
        plt.scatter(X[i,0],X[i,1], color=color)
    plt.title(f'Iteration : {j+1}')
    plt.xlabel('x1')
    plt.ylabel('x2')
plt.show()

### K-means 사이킷런

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.cluster import KMeans
 
kmeans = KMeans(n_clusters=3, init=init_center)
kmeans.fit(X)
labels = kmeans.labels_

In [ ]:
fig = plt.figure(figsize=(7,7))
fig.set_facecolor('white')
for i, label in enumerate(labels):
    if label == 0:
        color = 'blue'
    elif label ==1:
        color = 'red'
    else:
        color = 'green'
    plt.scatter(X[i,0],X[i,1], color=color)
    
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

## DBSCAN

In [ ]:
import pandas as pd
from sklearn import datasets
iris = datasets.load_iris()

labels = pd.DataFrame(iris.target)
labels.columns=['labels']
data = pd.DataFrame(iris.data)
data.columns=['Sepal length','Sepal width','Petal length','Petal width']
data = pd.concat([data,labels],axis=1)

data.head()

In [ ]:
feature = data[ ['Sepal length','Sepal width','Petal length','Petal width']]
feature.head()

In [ ]:
from sklearn.cluster import DBSCAN
import matplotlib.pyplot  as plt
import seaborn as sns

In [ ]:
model = DBSCAN(min_samples=6)
predict = pd.DataFrame(model.fit_predict(feature))
predict.columns=['predict']

In [ ]:
r = pd.concat([feature,predict],axis=1)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
# scatter plot
fig = plt.figure( figsize=(6,6))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(r['Sepal length'],r['Sepal width'],r['Petal length'],c=r['predict'],alpha=0.5)
ax.set_xlabel('Sepal lenth')
ax.set_ylabel('Sepal width')
ax.set_zlabel('Petal length')
plt.show()

In [ ]:
ct = pd.crosstab(data['labels'],r['predict'])
print (ct)